This notebook is created using keras
deep learning methods to train CNN with transfer learning(feature extraction).


---


I have used dataset from [kaggle](https://www.kaggle.com/datasets/jessicali9530/celeba-dataset) which contains 200k images of celebrities all around the world and it has also pre


---

# New Section
Due to limitations of
 my CPU i have not used all of the data but tried to learn concept of functional API by coding.


In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d jessicali9530/celeba-dataset

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
License(s): other
100% 1.33G/1.33G [00:08<00:00, 185MB/s]
100% 1.33G/1.33G [00:08<00:00, 167MB/s]


In [ ]:
!unzip celeba-dataset.zip -d ./celeba

# Data Preprocessing

In [5]:
df = pd.read_csv('./celeba/list_attr_celeba.csv')

In [6]:
df.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [7]:
df = df.replace(-1,0)

In [8]:
df.shape

(202599, 41)

In [24]:
train_df = df.sample(frac=0.001, random_state=42)
test_df = df.sample(frac=0.001, random_state=42)

In [25]:
train_df.shape, test_df.shape

((203, 41), (203, 41))

# Data augmentation


In [26]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [27]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory='./celeba/img_align_celeba/img_align_celeba',
    x_col='image_id',
    y_col=['Male', 'Young'],
    target_size=(224, 224),
    batch_size=32,
    class_mode = 'raw'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory='./celeba/img_align_celeba/img_align_celeba',
    x_col='image_id',
    y_col=['Male', 'Young'],
    target_size=(224, 224),
    batch_size=32,
    class_mode = 'raw'
)

Found 203 validated image filenames.
Found 203 validated image filenames.


In [28]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model

# Pre trained model
Here we are using VGG16 for our training, we have only taken convolution layer part and then we will train our neural network on top of that

In [29]:
VGG_16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [30]:
VGG_16.trainable = False

# creating new architecture on top of freezed VGG

In [31]:
last_layer_conv = VGG_16.output

In [32]:
flatten = Flatten()(last_layer_conv)
dense1 = Dense(128, activation='relu')(flatten)

predictions = Dense(2, activation='sigmoid', name='attributes')(dense1)

model = Model(inputs=VGG_16.input, outputs=predictions)

In [33]:
model = Model(inputs=VGG_16.input, outputs=predictions)

In [34]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attributes (Dense)              │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,338 (68.38 MB)

 Trainable params: 3,211,650 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from keras.utils import plot_model
plot_model(model)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Here i am using less data and also less epochs because of CPU limitations but if you have better computer you can increase the training and get better accuracy on both sets

In [37]:
model.fit(train_generator, epochs=5, validation_data=test_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 246s 38s/step - accuracy: 0.5345 - loss: 1.7387 - val_accuracy: 0.4631 - val_loss: 0.6537
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 250s 37s/step - accuracy: 0.5635 - loss: 0.6892 - val_accuracy: 0.7931 - val_loss: 0.5970
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 241s 37s/step - accuracy: 0.6852 - loss: 0.6268 - val_accuracy: 0.4631 - val_loss: 0.5253
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 263s 41s/step - accuracy: 0.4939 - loss: 0.5549 - val_accuracy: 0.4680 - val_loss: 0.5204
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 241s 37s/step - accuracy: 0.4544 - loss: 0.5272 - val_accuracy: 0.4631 - val_loss: 0.4879


We can also use fine tunig also by just unfreezing some last layers of our pretrained models and get more accurate results.